<a href="https://colab.research.google.com/github/Singh-AP/Intracranial-Aneurysm/blob/main/Densepoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!git clone https://github.com/Yochengliu/DensePoint.git
!cp -R DensePoint/* .
!mkdir build && cd build
!cmake . && make 
# !pip install torch==1.7.
# !pip install --upgrade torchvision
# !pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl


fatal: destination path 'DensePoint' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- Configuring done
-- Generating done
-- Build files have been written to: /content
[ 20%] Building NVCC (Device) object CMakeFiles/cuda_compile_1.dir/utils/csrc/cuda_compile_1_generated_ball_query_gpu.cu.o
[ 40%] Building NVCC (Device) object CMakeFiles/cuda_compile_1.dir/utils/csrc/cuda_compile_1_generated_group_points_gpu.cu.o
[ 60%] Building NVCC (Device) object CMakeFiles/cuda_compile_1.dir/utils/csrc/cuda_compile_1_generated_interpolate_gpu.cu.o
[ 80%] Building NVCC (Device) object CMakeFiles/cuda_compile_1.dir/utils/csrc/cuda_compile_1_generated_sampling_gpu.cu.o
[100%] Generating utils/_ext/pointnet2/_pointnet2.so
Traceback (most recent call last):
  File "/content/utils/build_ffi.py", line 4, in <module>
    from torch.utils.ffi import create_extension
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/__init__.py", line 1, in <mod

In [18]:
# !pip install torch-encoding==0.3.1
# !nvidia-smi



Thu Dec  3 13:31:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    33W / 250W |      1MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import os
import glob
import tempfile
import shutil
from functools import wraps, reduce
from string import Template
import torch
import torch.cuda
from torch._utils import _accumulate

try:
	import cffi
except ImportError:
	raise ImportError("torch.utils.ffi requires the cffi package")


if cffi.__version_info__ < (1, 4, 0):
	raise ImportError("torch.utils.ffi requires cffi version >= 1.4, but "
					  "got " + '.'.join(map(str, cffi.__version_info__)))


def _generate_typedefs():
	typedefs = []
	for t in ['Double', 'Float', 'Long', 'Int', 'Short', 'Char', 'Byte']:
		for lib in ['TH', 'THCuda']:
			for kind in ['Tensor', 'Storage']:
				python_name = t + kind
				if t == 'Float' and lib == 'THCuda':
					th_name = 'THCuda' + kind
				else:
					th_name = lib + t + kind
				th_struct = 'struct ' + th_name

				typedefs += ['typedef {} {};'.format(th_struct, th_name)]
				# We have to assemble a string here, because we're going to
				# do this lookup based on tensor.type(), which returns a
				# string (not a type object, as this code was before)
				python_module = 'torch.cuda' if lib == 'THCuda' else 'torch'
				python_class = python_module + '.' + python_name
				_cffi_to_torch[th_struct] = python_class
				_torch_to_cffi[python_class] = th_struct
	return '\n'.join(typedefs) + '\n'
_cffi_to_torch = {}
_torch_to_cffi = {}
_typedefs = _generate_typedefs()


PY_MODULE_TEMPLATE = Template("""
from torch.utils.ffi import _wrap_function
from .$cffi_wrapper_name import lib as _lib, ffi as _ffi

__all__ = []
def _import_symbols(locals):
	for symbol in dir(_lib):
		fn = getattr(_lib, symbol)
		if callable(fn):
			locals[symbol] = _wrap_function(fn, _ffi)
		else:
			locals[symbol] = fn
		__all__.append(symbol)

_import_symbols(locals())
""")


def _setup_wrapper(with_cuda):
	here = os.path.abspath(os.path.dirname(__file__))
	lib_dir = os.path.join(here, '..', '..', 'lib')
	include_dirs = [
		os.path.join(lib_dir, 'include'),
		os.path.join(lib_dir, 'include', 'TH'),
	]

	wrapper_source = '#include <TH/TH.h>\n'
	if with_cuda:
		import torch.cuda
		wrapper_source += '#include <THC/THC.h>\n'
		if os.sys.platform == 'win32':
			cuda_include_dirs = glob.glob(os.getenv('CUDA_PATH', '') + '/include')
			cuda_include_dirs += glob.glob(os.getenv('NVTOOLSEXT_PATH', '') + '/include')
		else:
			cuda_include_dirs = glob.glob('/usr/local/cuda/include')
			cuda_include_dirs += glob.glob('/Developer/NVIDIA/CUDA-*/include')
		include_dirs.append(os.path.join(lib_dir, 'include', 'THC'))
		include_dirs.extend(cuda_include_dirs)
	return wrapper_source, include_dirs


def _create_module_dir(base_path, fullname):
	module, _, name = fullname.rpartition('.')
	if not module:
		target_dir = name
	else:
		target_dir = reduce(os.path.join, fullname.split('.'))
	target_dir = os.path.join(base_path, target_dir)
	try:
		os.makedirs(target_dir)
	except os.error:
		pass
	for dirname in _accumulate(fullname.split('.'), os.path.join):
		init_file = os.path.join(base_path, dirname, '__init__.py')
		open(init_file, 'a').close()  # Create file if it doesn't exist yet
	return name, target_dir


def _build_extension(ffi, cffi_wrapper_name, target_dir, verbose):
	try:
		tmpdir = tempfile.mkdtemp()
		ext_suf = '.pyd' if os.sys.platform == 'win32' else '.so'
		libname = cffi_wrapper_name + ext_suf
		outfile = ffi.compile(tmpdir=tmpdir, verbose=verbose, target=libname)
		shutil.copy(outfile, os.path.join(target_dir, libname))
	finally:
		shutil.rmtree(tmpdir)


def _make_python_wrapper(name, cffi_wrapper_name, target_dir):
	py_source = PY_MODULE_TEMPLATE.substitute(name=name,
											  cffi_wrapper_name=cffi_wrapper_name)
	with open(os.path.join(target_dir, '__init__.py'), 'w') as f:
		f.write(py_source)


def create_extension(name, headers, sources, verbose=True, with_cuda=False,
					 package=False, relative_to='.', **kwargs):
	base_path = os.path.abspath(os.path.dirname(relative_to))
	name_suffix, target_dir = _create_module_dir(base_path, name)
	if not package:
		cffi_wrapper_name = '_' + name_suffix
	else:
		cffi_wrapper_name = (name.rpartition('.')[0] +
							 '.{0}._{0}'.format(name_suffix))

	wrapper_source, include_dirs = _setup_wrapper(with_cuda)
	include_dirs.extend(kwargs.pop('include_dirs', []))

	if os.sys.platform == 'win32':
		library_dirs = glob.glob(os.getenv('CUDA_PATH', '') + '/lib/x64')
		library_dirs += glob.glob(os.getenv('NVTOOLSEXT_PATH', '') + '/lib/x64')

		here = os.path.abspath(os.path.dirname(__file__))
		lib_dir = os.path.join(here, '..', '..', 'lib')

		library_dirs.append(os.path.join(lib_dir))
	else:
		library_dirs = []
	library_dirs.extend(kwargs.pop('library_dirs', []))

	if isinstance(headers, str):
		headers = [headers]
	all_headers_source = ''
	for header in headers:
		with open(os.path.join(base_path, header), 'r') as f:
			all_headers_source += f.read() + '\n\n'

	ffi = cffi.FFI()
	sources = [os.path.join(base_path, src) for src in sources]
	# NB: TH headers are C99 now
	kwargs['extra_compile_args'] = ['-std=c99'] + kwargs.get('extra_compile_args', [])
	ffi.set_source(cffi_wrapper_name, wrapper_source + all_headers_source,
				   sources=sources,
				   include_dirs=include_dirs,
				   library_dirs=library_dirs, **kwargs)
	ffi.cdef(_typedefs + all_headers_source)

	_make_python_wrapper(name_suffix, '_' + name_suffix, target_dir)

	def build():
		_build_extension(ffi, cffi_wrapper_name, target_dir, verbose)
	ffi.build = build
	return ffi

def _wrap_function(function, ffi):
	@wraps(function)
	def safe_call(*args, **kwargs):
		args = tuple(ffi.cast(_torch_to_cffi.get(arg.type(), 'void') + '*', arg._cdata)
					 if isinstance(arg, torch.Tensor) or torch.is_storage(arg)
					 else arg
					 for arg in args)
		args = (function,) + args
		result = torch._C._safe_call(*args, **kwargs)
		if isinstance(result, ffi.CData):
			typeof = ffi.typeof(result)
			if typeof.kind == 'pointer':
				cdata = int(ffi.cast('uintptr_t', result))
				cname = typeof.item.cname
				if cname in _cffi_to_torch:
					# TODO: Maybe there is a less janky way to eval
					# off of this
					return eval(_cffi_to_torch[cname])(cdata=cdata)
		return result
	return safe_call

In [17]:
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_sched
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
import os
from torchvision import transforms
from models import DensePointCls_L6 as DensePoint
from data import ModelNet40Cls
import utils.pytorch_utils as pt_utils
import utils.pointnet2_utils as pointnet2_utils
import data.data_utils as d_utils
import argparse
import random
import yaml

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)            
torch.cuda.manual_seed(seed)       
torch.cuda.manual_seed_all(seed)   

parser = argparse.ArgumentParser(description='DensePoint Shape Classification Training')
parser.add_argument('--config', default='cfgs/config_cls.yaml', type=str)

def main():
    args = parser.parse_args()
    with open(args.config) as f:
        config = yaml.load(f)
    print("\n**************************")
    for k, v in config['common'].items():
        setattr(args, k, v)
        print('\n[%s]:'%(k), v)
    print("\n**************************\n")
    
    try:
        os.makedirs(args.save_path)
    except OSError:
        pass
    
    train_transforms = transforms.Compose([
        d_utils.PointcloudToTensor()
    ])
    test_transforms = transforms.Compose([
        d_utils.PointcloudToTensor()
    ])
    
    train_dataset = ModelNet40Cls(num_points = args.num_points, root = args.data_root, transforms=train_transforms)
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=args.batch_size,
        shuffle=True, 
        num_workers=int(args.workers), 
        pin_memory=True
    )

    test_dataset = ModelNet40Cls(num_points = args.num_points, root = args.data_root, transforms=test_transforms, train=False)
    test_dataloader = DataLoader(
        test_dataset, 
        batch_size=args.batch_size,
        shuffle=False, 
        num_workers=int(args.workers), 
        pin_memory=True
    )
    
    model = DensePoint(num_classes = args.num_classes, input_channels = args.input_channels, use_xyz = True)
    model.cuda()
    optimizer = optim.Adam(
        model.parameters(), lr=args.base_lr, weight_decay=args.weight_decay)

    lr_lbmd = lambda e: max(args.lr_decay**(e // args.decay_step), args.lr_clip / args.base_lr)
    bnm_lmbd = lambda e: max(args.bn_momentum * args.bn_decay**(e // args.decay_step), args.bnm_clip)
    lr_scheduler = lr_sched.LambdaLR(optimizer, lr_lbmd)
    bnm_scheduler = pt_utils.BNMomentumScheduler(model, bnm_lmbd)
    
    if args.checkpoint is not '':
        model.load_state_dict(torch.load(args.checkpoint))
        print('Load model successfully: %s' % (args.checkpoint))

    criterion = nn.CrossEntropyLoss()
    num_batch = len(train_dataset)/args.batch_size
    
    # training
    train(train_dataloader, test_dataloader, model, criterion, optimizer, lr_scheduler, bnm_scheduler, args, num_batch)
    

def train(train_dataloader, test_dataloader, model, criterion, optimizer, lr_scheduler, bnm_scheduler, args, num_batch):
    PointcloudScaleAndTranslate = d_utils.PointcloudScaleAndTranslate()   # initialize augmentation
    global g_acc 
    g_acc = 0.91    # only save the model whose acc > 0.91
    batch_count = 0
    model.train()
    for epoch in range(args.epochs):
        for i, data in enumerate(train_dataloader, 0):
            if lr_scheduler is not None:
                lr_scheduler.step(epoch)
            if bnm_scheduler is not None:
                bnm_scheduler.step(epoch-1)
            points, target = data
            points, target = points.cuda(), target.cuda()
            points, target = Variable(points), Variable(target)
            
            # farthest point sampling
            fps_idx = pointnet2_utils.furthest_point_sample(points, 1200)  # (B, npoint)
            fps_idx = fps_idx[:, np.random.choice(1200, args.num_points, False)]
            points = pointnet2_utils.gather_operation(points.transpose(1, 2).contiguous(), fps_idx).transpose(1, 2).contiguous()  # (B, N, 3)
            
            # augmentation
            points.data = PointcloudScaleAndTranslate(points.data)
            
            optimizer.zero_grad()
            
            pred = model(points)
            target = target.view(-1)
            loss = criterion(pred, target)
            loss.backward()
            optimizer.step()
            if i % args.print_freq_iter == 0:
                print('[epoch %3d: %3d/%3d] \t train loss: %0.6f \t lr: %0.5f' %(epoch+1, i, num_batch, loss.data.clone(), lr_scheduler.get_lr()[0]))
            batch_count += 1
            
            # validation in between an epoch
            if args.evaluate and batch_count % int(args.val_freq_epoch * num_batch) == 0:
                validate(test_dataloader, model, criterion, args, batch_count)


def validate(test_dataloader, model, criterion, args, iter): 
    global g_acc
    model.eval()
    losses, preds, labels = [], [], []
    for j, data in enumerate(test_dataloader, 0):
        points, target = data
        points, target = points.cuda(), target.cuda()
        points, target = Variable(points, volatile=True), Variable(target, volatile=True)
        
        # farthest point sampling
        fps_idx = pointnet2_utils.furthest_point_sample(points, args.num_points)  # (B, npoint)
        # fps_idx = fps_idx[:, np.random.choice(1200, args.num_points, False)]
        points = pointnet2_utils.gather_operation(points.transpose(1, 2).contiguous(), fps_idx).transpose(1, 2).contiguous()

        pred = model(points)
        target = target.view(-1)
        loss = criterion(pred, target)
        losses.append(loss.data.clone())
        _, pred_choice = torch.max(pred.data, -1)
        
        preds.append(pred_choice)
        labels.append(target.data)
        
    preds = torch.cat(preds, 0)
    labels = torch.cat(labels, 0)
    acc = (preds == labels).sum() / labels.numel()
    print('\nval loss: %0.6f \t acc: %0.6f\n' %(np.array(losses).mean(), acc))
    if acc > g_acc:
        g_acc = acc
        torch.save(model.state_dict(), '%s/cls_iter_%d_acc_%0.6f.pth' % (args.save_path, iter, acc))
    model.train()
    
if __name__ == "__main__":
    main()

RuntimeError: ignored